In [1]:
import glob
import os
import sys
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('max_colwidth', 50)
os.chdir('/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling')

In [29]:
# setting input and output with jupyter notebook in context 
if 'ipykernel_launcher.py' in sys.argv[0]:
    output_prefix = 'results/samplesheets/post-hicpro/human_t2t_biorep_merged.peaks_files.samplesheet'
# if not using jupyter notebook
else:
    input_fn = sys.argv[1]
    output_prefix = sys.argv[2]

In [30]:
# list of GSE IDs of processed ChIP-seq samples, or GSE IDs without any ChIP-seq samples
# this list is used to determine which HiChIP samples do not have matching ChIP-seq data

# Note: the GSE IDs are those corresponding to the matching HiChIP data, not necessarily the IDs with ChIP-seq data themselves
matching_chipseq_human = ["GSE136090", "GSE116193", "GSE105028", "GSE128106", "GSE147646"]
matching_chipseq_human.extend(["GSE179544", "GSE108869", "GSE117888", "GSE116869", "GSE116495", "GSE115492"])
matching_chipseq_human.extend(["GSE111537", "GSE131054", "GSE131651", "GSE136629", "GSE137849", "GSE149420"])
matching_chipseq_human.extend(["GSE151001", "GSE154513", "GSE156650", "GSE159985", "GSE157107", "GSE157381"])
matching_chipseq_human.extend(["GSE165207", "GSE173843", "GSE179666", "GSE180194", "GSE97585", "GSE166232"])
matching_chipseq_human.extend(["GSE165303", "phs001703v3p1", "phs001703v4p1"])

no_matching_chipseq_human = ["GSE178598", "GSE101498", "GSE147854", "GSE151193", "GSE179755", "GSE173699"]
no_matching_chipseq_human.extend(["GSE92881", "GSE100856", "GSE111930", "GSE115524", "GSE115896", "GSE116872"])
no_matching_chipseq_human.extend(["GSE116875", "GSE126792", "GSE137168", "GSE137906", "GSE137936", "GSE139466"])
no_matching_chipseq_human.extend(["GSE148958", "GSE152900", "GSE155184", "GSE155328", "GSE156772"])
no_matching_chipseq_human.extend(["GSE158642", "GSE168881", "GSE171591", "GSE173871", "GSE181971", "GSE183797"])
no_matching_chipseq_human.extend(["GSE188380", "GSE196235", "GSE80820", "GSE150906", "GSE133227", "GSE99519", "GSE120294"])
no_matching_chipseq_human.extend(["GSE147672"])

matching_chipseq_mouse = ["GSE112717", "GSE141113", "GSE110898", "GSE141847", "GSE142004", "GSE159629"]
matching_chipseq_mouse.extend(["GSE178344", "GSE147919"])

no_matching_chipseq_mouse = ["GSE101498", "GSE148691", "GSE112176", "GSE113339", "GSE115524"]
no_matching_chipseq_mouse.extend(["GSE121671", "GSE126362", "GSE135296", "GSE148691", "GSE150536"])
no_matching_chipseq_mouse.extend(["GSE150906", "GSE153884", "GSE157666", "GSE160656", "GSE162617"])
no_matching_chipseq_mouse.extend(["GSE189442", "GSE192387", "GSE193079", "GSE194068", "GSE80820"])
no_matching_chipseq_mouse.extend(["GSE99519", "GSE145793", "GSE166177"])

processed_gse_ids = matching_chipseq_human + no_matching_chipseq_human + matching_chipseq_mouse + no_matching_chipseq_mouse

#### Function to generate pandas dataframe

In [31]:
def read_chipseq_files(file):
    peak_data = {}
    count = 0
    with open(file) as fr:
        for line in fr:
            count = count + 1
    return(count)

In [32]:
# Create a dataframe with the following columns:
# 1) Sample Name
# 2) HiC-Pro .allValidPairs file
# 3) Matching ChIP-seq peaks (if it exists)
# 4) Matching FitHiChIP peaks (if it has been processed yet)

def generate_df():
    
    data = []
    info = []
    hicpro_paths = glob.glob('ref_genome/biorep_merged/results/hicpro/*Homo*/*.allValidPairs')
    
    # iterate through each HiC-Pro output directory
    for fn in hicpro_paths:
        
        sample_name = fn.split('/')[4]
        sample_name_chipseq = sample_name[:-13] + "b1"
        sample_name_chipseq_general = sample_name[:-13] + "b"
        # hicpro validpairs file
        hicpro_path = '/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/'+ fn
        
        chipseq_peak_file="Not_processed_or_not_available"
        chipline_path="/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/ref_genome/results/peaks"
        
        # for in-house HiChIP samples with pre-generated peaks
    
        # update hicpro sample names to match chipline sample names for pieQTL/NCM samples            
        if 'phs001703v' in sample_name:
            if 'phs001703v3' in sample_name:
                cell_type = sample_name.split('.')[0].split('_1')[0]
                sample_name_chipseq = cell_type + "_merged_donors_hg38.phs001703v3p1.Homo_Sapiens.H3K27ac.b1"
            if 'phs001703v4' in sample_name:
                cell_type = sample_name.split('.')[0].split('_1')[0]
                sample_name_chipseq = cell_type + "_merged_donors_hg38.phs001703v4p1.Homo_Sapiens.H3K27ac.b1"
                
                
        # check whether a folder matching sample_name exists in the merged_chipline folder
        merged_chipline_path = chipline_path + "/merged_chipline/" + sample_name_chipseq
        count = 0
        if os.path.exists(merged_chipline_path):
            sample_info = []
            sample_info.append(sample_name)
            chipseq_peak_file = merged_chipline_path + "/FINAL_IDR_Peaks_FDR0.05.txt"
            sample_info.append(read_chipseq_files(chipseq_peak_file))
            chipline_path += "/chipline/" + sample_name_chipseq_general + "*/MACS2_Ext*/*.macs2_peaks.narrowPeak_Q0.01filt"
            replicates = glob.glob(chipline_path)
            max_rep_peaks = -1
            max_rep = ""
            for rep in replicates:
                num_peaks = int(read_chipseq_files(rep))
                sample_info.append(num_peaks)
                if num_peaks > max_rep_peaks:
                    max_rep_peaks = num_peaks
                    max_rep = rep
            chipseq_peak_file = max_rep
            sample_info.append(max_rep_peaks)
            info.append(sample_info)
            count = 0
            with open(chipseq_peak_file) as c:
                for line in c:
                    count = count + 1

        # check whether a folder matching sample_name exists in the chipline folder
        # IMPORTANT: Replace the last character of the sample name with 1 (so it is .b1 to match with the ChIP-seq loops)
        chipline_path += "/chipline/" + sample_name_chipseq + "/MACS2_Ext*/*.macs2_peaks.narrowPeak_Q0.01filt"
        if chipseq_peak_file == "Not_processed_or_not_available" and len(glob.glob(chipline_path)) > 0:
            chipseq_peak_file = glob.glob(chipline_path)[0]

        # check if the matching ChIP-seq sample does not exist
        if any(gse_id in sample_name_chipseq for gse_id in processed_gse_ids) and not os.path.exists(chipseq_peak_file):
            chipseq_peak_file = "Not_available"

        # fix path if assigned chip-seq path does not contain peak calls
        ## NOTE: REMOVE FIRST CONDITION FOR ADDITION OF CHIPSEQ PEAKS TO FILE
        if chipseq_peak_file!="Not_processed_or_not_available" and chipseq_peak_file != "Not_available" and os.path.getsize(chipseq_peak_file) == 0:
            print("sample:", sample_name)
            print("old path:", chipseq_peak_file)

            new_path = ''
            if "merged_chipline" in chipseq_peak_file:
                new_path = glob.glob(chipline_path)[0]
                if os.path.getsize(new_path) == 0:
                    print("rep 1 was empty")
                    new_path = glob.glob(chipline_path)[0].replace('b1', 'b2')

            if new_path == '':
                if os.path.getsize('/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/ref_genome/results/peaks/chipline/{sn}/MACS2_Ext_with_Control/{sn}.macs2_peaks.narrowPeak_Q0.05filt'.format(sn = sample_name_chipseq)) == 0:
                    print("reg chipline 0.05 filt is also empty")
                    new_path = "Not_available"
                else:
                    print("reg chipline 0.05 filt is NOT empty")
                    new_path = '/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/ref_genome/results/peaks/chipline/{sn}/MACS2_Ext_with_Control/{sn}.macs2_peaks.narrowPeak_Q0.05filt'.format(sn = sample_name_chipseq)

            print("new path:", new_path)
            print("new path size:", os.path.getsize(new_path))
            print(os.path.exists(new_path))
            print()

            chipseq_peak_file = new_path
            
        # check whether a folder matching the sample name exists in the fithichip folder
        fithichip_peak_file = "Not_processed"
        fithichip_peak_path = 'ref_genome/biorep_merged/results/peaks/fithichip/{sn}/MACS2_ExtSize/out_macs2_peaks.narrowPeak'.format(sn = sample_name)
        if os.path.exists(fithichip_peak_path):
            fithichip_peak_file = '/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/' + fithichip_peak_path
        
        # check whether a folder matching the sample name exists in the hichip-peaks folder
        hichippeaks_peak_file = "Not_processed"
        hichippeaks_path = 'ref_genome/biorep_merged/results/peaks/hichip-peaks/{sn}/out_peaks.bed'.format(sn = sample_name)
        if os.path.exists(hichippeaks_path):
            hichippeaks_peak_file = '/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/' + hichippeaks_path

        # add all the information for this sample to the 2D array
        sample_data = [sample_name, hicpro_path, hichippeaks_peak_file, fithichip_peak_file, chipseq_peak_file, count]
        
        data.append(sample_data)
        
    return pd.DataFrame(data)
    

#### Display the pandas dataframe

In [33]:
df = generate_df()

In [34]:
df = df.sort_values(by=0).reset_index(drop=True)

In [35]:
len(df.loc[df[4].str.contains("/mnt")])

51

In [36]:
df

,0,1,2,3,4,5
0,A673-siCT-Dh1-72h-D347-D362.GSE156650.Homo_Sap...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,0
1,A673-siSA2-Dh6-72h-D347-D362.GSE156650.Homo_Sa...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,0
2,A673-siSA2-Dh8-72h-D347-D362.GSE156650.Homo_Sa...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,0
3,A673_SA1m1.GSE133227.Homo_Sapiens.CTCF.biorep_...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_available,0
4,A673_SA2m1.GSE133227.Homo_Sapiens.CTCF.biorep_...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_available,0
5,A673_SA2r.GSE133227.Homo_Sapiens.CTCF.biorep_m...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_available,0
6,A673_SA2r.GSE133227.Homo_Sapiens.H3K27ac.biore...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_available,0
7,A673_WT.GSE133227.Homo_Sapiens.CTCF.biorep_merged,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_available,0
8,A673_WT.GSE133227.Homo_Sapiens.H3K27ac.biorep_...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_available,0
9,CAR-T-CD19-D10.GSE168881.Homo_Sapiens.H3K27ac....,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_available,0


In [37]:
df.columns = ["std_sample_name", "validpairs_path", "hichip_peaks_path", "fithichip_path", "chipseq_path", "num"]
df = df.sort_values(by=["std_sample_name"], ascending=True).reset_index(drop=True)
df

,std_sample_name,validpairs_path,hichip_peaks_path,fithichip_path,chipseq_path,num
0,A673-siCT-Dh1-72h-D347-D362.GSE156650.Homo_Sap...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,0
1,A673-siSA2-Dh6-72h-D347-D362.GSE156650.Homo_Sa...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,0
2,A673-siSA2-Dh8-72h-D347-D362.GSE156650.Homo_Sa...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,0
3,A673_SA1m1.GSE133227.Homo_Sapiens.CTCF.biorep_...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_available,0
4,A673_SA2m1.GSE133227.Homo_Sapiens.CTCF.biorep_...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_available,0
5,A673_SA2r.GSE133227.Homo_Sapiens.CTCF.biorep_m...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_available,0
6,A673_SA2r.GSE133227.Homo_Sapiens.H3K27ac.biore...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_available,0
7,A673_WT.GSE133227.Homo_Sapiens.CTCF.biorep_merged,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_available,0
8,A673_WT.GSE133227.Homo_Sapiens.H3K27ac.biorep_...,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_available,0
9,CAR-T-CD19-D10.GSE168881.Homo_Sapiens.H3K27ac....,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_processed,/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip...,Not_available,0


In [38]:
len(df.loc[df['chipseq_path'].str.contains("/mnt")])

51

In [39]:
## want 237; 32
num_samples_chipseq = 0
samples_chipseq = []
for index, row in df.iterrows():
    if '/mnt/' in row['chipseq_path']:
        num_samples_chipseq = num_samples_chipseq + 1
        study = row['std_sample_name'].split('.')[1]
        if study not in samples_chipseq:
            samples_chipseq.append(study)
print("num samples with chipseq:", num_samples_chipseq)
print("num studies with chipseq:", len(samples_chipseq))
print((' \n').join(samples_chipseq))

num samples with chipseq: 51
num studies with chipseq: 9
GSE156650 
phs001703v3p1 
GSE137849 
GSE108869 
GSE97585 
GSE131651 
phs001703v4p1 
GSE111537 
GSE116495


In [40]:
print((' \n').join(matching_chipseq_human))

GSE136090 
GSE116193 
GSE105028 
GSE128106 
GSE147646 
GSE179544 
GSE108869 
GSE117888 
GSE116869 
GSE116495 
GSE115492 
GSE111537 
GSE131054 
GSE131651 
GSE136629 
GSE137849 
GSE149420 
GSE151001 
GSE154513 
GSE156650 
GSE159985 
GSE157107 
GSE157381 
GSE165207 
GSE173843 
GSE179666 
GSE180194 
GSE97585 
GSE166232 
GSE165303 
phs001703v3p1 
phs001703v4p1


#### Output the dataframe as a tsv file

In [41]:
without_header_output = '{}.without_header.tsv'.format(output_prefix)
df.to_csv(without_header_output, header=False, index=False, sep='\t')

In [26]:
def generate_df_merged_validpairs():
    
    data = []
    hicpro_paths = glob.glob('results/pieqtl_ncm_rep_combined_donorwise/validpairs/*/*.allValidPairs')
    
    # iterate through each HiC-Pro output directory
    for fn in hicpro_paths:
        
        sample_name = fn.split('/')[3]
        sample_name_chipseq = sample_name[:-12] + "1"
        print(sample_name_chipseq)
        # hicpro validpairs file
        hicpro_path = '/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/'+ fn
        
        chipseq_peak_file="Not_processed_or_not_available"
        chipline_path="/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks"
        
        # for in-house HiChIP samples with pre-generated peaks
    
        # update hicpro sample names to match chipline sample names for pieQTL/NCM samples            
        if 'phs001703v' in sample_name:
            if 'phs001703v3' in sample_name:
                cell_type = sample_name.split('.')[0].split('_1')[0]
                sample_name_chipseq = cell_type + "_merged_donors_hg38.phs001703v3p1.Homo_Sapiens.H3K27ac.b1"
            if 'phs001703v4' in sample_name:
                cell_type = sample_name.split('.')[0].split('_1')[0]
                sample_name_chipseq = cell_type + "_merged_donors_hg38.phs001703v4p1.Homo_Sapiens.H3K27ac.b1"
                
                
        # check whether a folder matching sample_name exists in the merged_chipline folder
        merged_chipline_path = chipline_path + "/merged_chipline/" + sample_name_chipseq
        if os.path.exists(merged_chipline_path):
            chipseq_peak_file = merged_chipline_path + "/FINAL_IDR_Peaks_FDR0.05.txt"

        # check whether a folder matching sample_name exists in the chipline folder
        # IMPORTANT: Replace the last character of the sample name with 1 (so it is .b1 to match with the ChIP-seq loops)
        chipline_path += "/chipline/" + sample_name_chipseq + "/MACS2_Ext*/" + sample_name_chipseq + ".macs2_peaks.narrowPeak_Q0.01filt"
        if chipseq_peak_file == "Not_processed_or_not_available" and len(glob.glob(chipline_path)) > 0:
            chipseq_peak_file = glob.glob(chipline_path)[0]

        # check if the matching ChIP-seq sample does not exist
        if any(gse_id in sample_name_chipseq for gse_id in processed_gse_ids) and not os.path.exists(chipseq_peak_file):
            chipseq_peak_file = "Not_available"

        # fix path if assigned chip-seq path does not contain peak calls
        ## NOTE: REMOVE FIRST CONDITION FOR ADDITION OF CHIPSEQ PEAKS TO FILE
        if chipseq_peak_file!="Not_processed_or_not_available" and chipseq_peak_file != "Not_available" and os.path.getsize(chipseq_peak_file) == 0:
            print("sample:", sample_name)
            print("old path:", chipseq_peak_file)

            new_path = ''
            if "merged_chipline" in chipseq_peak_file:
                new_path = glob.glob(chipline_path)[0]
                if os.path.getsize(new_path) == 0:
                    print("rep 1 was empty")
                    new_path = glob.glob(chipline_path)[0].replace('b1', 'b2')

            if new_path == '':
                if os.path.getsize('/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline/{sn}/MACS2_Ext_with_Control/{sn}.macs2_peaks.narrowPeak_Q0.05filt'.format(sn = sample_name_chipseq)) == 0:
                    print("reg chipline 0.05 filt is also empty")
                    new_path = "Not_available"
                else:
                    print("reg chipline 0.05 filt is NOT empty")
                    new_path = '/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/peaks/chipline/{sn}/MACS2_Ext_with_Control/{sn}.macs2_peaks.narrowPeak_Q0.05filt'.format(sn = sample_name_chipseq)

            print("new path:", new_path)
            print("new path size:", os.path.getsize(new_path))
            print(os.path.exists(new_path))
            print()

            chipseq_peak_file = new_path
            
        # check whether a folder matching the sample name exists in the fithichip folder
        fithichip_peak_file = "Not_processed"
        fithichip_peak_path = 'results/peaks/fithichip/{sn}/MACS2_ExtSize/out_macs2_peaks.narrowPeak'.format(sn = sample_name)
        if os.path.exists(fithichip_peak_path):
            fithichip_peak_file = '/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/' + fithichip_peak_path
        
        # check whether a folder matching the sample name exists in the hichip-peaks folder
        hichippeaks_peak_file = "Not_processed"
        hichippeaks_path = 'results/peaks/hichip-peaks/{sn}/out_peaks.bed'.format(sn = sample_name)
        if os.path.exists(hichippeaks_path):
            hichippeaks_peak_file = '/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/' + hichippeaks_path

        # add all the information for this sample to the 2D array
        sample_data = [sample_name, hicpro_path, hichippeaks_peak_file, fithichip_peak_file, chipseq_peak_file]
        
        data.append(sample_data)
        
    return pd.DataFrame(data)